# LangGraph Tutorial: Single Tool Execution

**Objective:** Understand the basic execution pattern: Agent → Tool → Response

**Pattern:**
```
User Query → Agent analyzes → Calls tool → Tool executes → Agent responds
```

## Setup

Rebuild the graph from Notebook 3.

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

✅ Environment loaded


In [20]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Args:
        amount: The amount to convert
        from_currency: Source currency code (USD, EUR, GBP, INR, JPY)
        to_currency: Target currency code (USD, EUR, GBP, INR, JPY)
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Args:
        principal: The loan amount
        annual_interest_rate: Annual interest rate as percentage
        tenure_months: Loan tenure in months
        currency: Currency code for display
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

✅ Tools defined


In [21]:
# Initialize LLM with tools
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    max_tokens=1024,
    project=os.getenv("GOOGLE_PROJECT_ID"),
    location=os.getenv("GOOGLE_REGION")
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")

✅ LLM initialized with tools


In [22]:
# Build graph
def call_llm(state: MessagesState):
    """Agent node that invokes the LLM"""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    """Router that decides next step"""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


---

## Example: Single Tool Call

**Query:** "What is 1000 USD in EUR?"

**Expected Flow:**
1. Agent receives query
2. Agent calls `currency_converter` tool
3. Tool returns conversion result
4. Agent formulates natural language response

In [23]:
# Create initial state
state = {"messages": [HumanMessage(content="What is 1000 USD in EUR?")]}

print("Query: What is 1000 USD in EUR?")
print("=" * 70)

Query: What is 1000 USD in EUR?


In [ ]:
# Execute the graph
result = app.invoke(state)

print("\n✅ Execution complete")
print(f"Total messages in conversation: {len(result['messages'])}")


✅ Execution complete
Total messages in conversation: 4


In [ ]:
result

{'messages': [HumanMessage(content='What is 1000 USD in EUR?', additional_kwargs={}, response_metadata={}, id='093293e4-c1ba-4dfa-a61c-f742df7a880f'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'currency_converter', 'arguments': '{"from_currency": "USD", "amount": 1000, "to_currency": "EUR"}'}, '__gemini_function_call_thought_signatures__': {'4bcb94fc-024e-40d2-bc54-09c05b95c96b': 'CpkDAY89a1/eY6W5ak3cnbpae+BX6kd9lvUvPwHs0pcQ1csqLWI9fIvvBec238CHRiPY+Sr5WADKSF2LD8Y/QbRP+F58574O8m2uQP0ln5c+wU65EWia/YtpMm7cG+CridXN5tnGzJzNMSn2k5HFudSumMyicGmPqFzYBiZwxf3wqfpIFUjQ9RSCWZ0BXor23SOqcFprqpp0zvERAoKyAg1G3b/MasawzWOKGKrNSkh8+lbvfgGL4eUMTzLZHDUdtEaUnhf8ayiuev61/wlkUgckjnd4gO6epw1yw1WgVYvxgSr8K0aGgxqelU5GLaZb3lClIm7oeUtz1bveMhqnRM7Ac/02kbGzbQ1C2Fsld6SSlM0QdzBtjkI2i7GZ9Jx+37zmJdXZ8TzJDOPu42bUhsA+V3mfMN9rxtqkpi4SkrKwJOVsG3Mo/2AJ32jC101BY6q7W7wrp+YZ/5BK3l/ANjzfxGmrHb6NYr3rDY40Q17isQpnrWttKXLD7qXoOBDPK9sJoqk+/NVky300PGfnN2lnSb44K/Ey+d8I7A=='}}, response_metadata={'finish_reaso

In [ ]:
result['messages'][1].tool_calls

[{'name': 'currency_converter',
  'args': {'from_currency': 'USD', 'amount': 1000, 'to_currency': 'EUR'},
  'id': '4bcb94fc-024e-40d2-bc54-09c05b95c96b',
  'type': 'tool_call'}]

In [ ]:
result['messages'][2]

ToolMessage(content='Conversion Result:\n  1,000.00 USD = 920.00 EUR\n  Exchange Rate: 1 USD = 0.9200 EUR', name='currency_converter', id='81276b1b-ec27-4346-9081-26183e4dc4ad', tool_call_id='4bcb94fc-024e-40d2-bc54-09c05b95c96b')

## Inspect Message Flow

Examine each message in the conversation to understand the execution.

In [ ]:
print("CONVERSATION FLOW:")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    if isinstance(msg, HumanMessage):
        print(f"\n[{i}] 👤 USER:")
        print(f"    {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 AGENT (calling tools):")
            for tc in msg.tool_calls:
                print(f"    → Tool: {tc['name']}")
                print(f"      Args: {tc['args']}")
        else:
            print(f"\n[{i}] 🤖 AGENT (final response):")
            print(f"    {msg.content}")
            
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL RESULT:")
        print(f"    {msg.content}")

CONVERSATION FLOW:

[1] 👤 USER:
    What is 1000 USD in EUR?

[2] 🤖 AGENT (calling tools):
    → Tool: currency_converter
      Args: {'from_currency': 'USD', 'amount': 1000, 'to_currency': 'EUR'}

[3] 🔧 TOOL RESULT:
    Conversion Result:
  1,000.00 USD = 920.00 EUR
  Exchange Rate: 1 USD = 0.9200 EUR

[4] 🤖 AGENT (final response):
    1000 USD is equal to 920 EUR.


## Message Type Analysis

Understanding the message types in the flow.

In [ ]:
print("MESSAGE TYPES:")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    msg_type = type(msg).__name__
    print(f"[{i}] {msg_type}")

print("\n" + "=" * 70)
print("PATTERN OBSERVED:")
print("=" * 70)
print("1. HumanMessage    → User's question")
print("2. AIMessage       → Agent decides to call tool")
print("3. ToolMessage     → Tool execution result")
print("4. AIMessage       → Agent's final response")

MESSAGE TYPES:
[1] HumanMessage
[2] AIMessage
[3] ToolMessage
[4] AIMessage

PATTERN OBSERVED:
1. HumanMessage    → User's question
2. AIMessage       → Agent decides to call tool
3. ToolMessage     → Tool execution result
4. AIMessage       → Agent's final response


## Extract Final Response

Get the agent's final answer to the user.

In [ ]:
final_response = result["messages"][-1].content

print("FINAL RESPONSE TO USER:")
print("=" * 70)
print(final_response)
print("=" * 70)

FINAL RESPONSE TO USER:
1000 USD is equal to 920 EUR.


## Test Another Single Tool Call

**Query:** "Calculate EMI for a 50000 USD loan at 7.5% for 36 months"

In [40]:
# Create new state
state2 = {
    "messages": [
        HumanMessage(content="Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
    ]
}

print("Query: Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
print("=" * 70)

Query: Calculate EMI for a 50000 USD loan at 7.5% for 36 months


In [ ]:
# Execute
result2 = app.invoke(state2)

In [ ]:
result2

{'messages': [HumanMessage(content='Calculate EMI for a 50000 USD loan at 7.5% for 36 months', additional_kwargs={}, response_metadata={}, id='2df31810-802c-4064-b8ff-61b621735412'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'emi_calculator', 'arguments': '{"tenure_months": 36, "annual_interest_rate": 7.5, "principal": 50000, "currency": "USD"}'}, '__gemini_function_call_thought_signatures__': {'8230c6ac-f806-4883-aaf7-50d0920c0e57': 'CooJAY89a183Zr6hp/7sVzYcemSf5fWIDDVfM7hJXAKOCuJ5DXY+O/G5e7My58kpfVSOi42qN+YbCCfYO0C2H8ldm56+Z5EUiLAra3hL4ulyngFnhIXDmT1QzKs6Hpv2dub8OK4TjZJRfetiycK7y6n9z++qshNuFIUaUpT8Yd3/uGDYyz0pRraLicrNVYgPSUloXqixwxAHTCuvO6c1knS4v/gb0ep2Qx5hI2jY5Ma+v+mBJW+LzUsi3H6JEPapGcqzt89M3W+ZdZm5OeQnWNWZ9I32vX6XIhkaMiWybLK7NPqWanYdz79zQWQ2EFiJX0rYH41WkprXCie7Zhozth4cQq9QSNdbrzOdC37E1UPwll11EswILlaWk4+oNpzjQ0mUZW+L/iA6HLfSYszTpzGJ5QRO5ozohEgL5onOyjZZ9iYf9MfRPmgJAtE8Pg9eSmkdZkpfqmz/aeeC+kTFtVLcF+YkG7+nW5FTEd4L4D24johqmHE5rTAoPYe+XPFhA4ypnKQhy+KF6AgThEcGX3

In [ ]:

# Display flow
print("\nCONVERSATION FLOW:")
print("=" * 70)

for i, msg in enumerate(result2["messages"], 1):
    if isinstance(msg, HumanMessage):
        print(f"\n[{i}] 👤 USER: {msg.content}")
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 AGENT: Calling {msg.tool_calls[0]['name']}")
        else:
            print(f"\n[{i}] 🤖 AGENT: {msg.content}")
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL: Executed successfully")


CONVERSATION FLOW:

[1] 👤 USER: Calculate EMI for a 50000 USD loan at 7.5% for 36 months

[2] 🤖 AGENT: Calling emi_calculator

[3] 🔧 TOOL: Executed successfully

[4] 🤖 AGENT: [{'type': 'text', 'text': 'Your EMI for a 50000 USD loan at 7.5% for 36 months would be 1,555.31 USD per month. The total interest paid would be 5,991.19 USD, and the total payment would be 55,991.19 USD.', 'extras': {'signature': 'CpQDAY89a18FLW7nlcBmGb4i6x+qa7XApgJKajpvJEQw9CWQ64m39H01JEOzaFUMB/KBM9ro0PJC4Z6Nm3k+wDnmItWVenrpMhjYIh81480tvOp3/lWV3UpF11KEFKYDyULnVK6+rTj3ouZaPsfDuXGfd/gh1tV0xL7MRcIDtfbBiz2W5SvqwOs9RwOMEzD3JndxFItYooWOm1tnmCrP7+k8RjtrWAxN8cljhCJHH4frx00v0GH96PKEdvVSIqrzeaItuOz4fpHreRFjB4+4ea9En+dhnHfkI14tGSE7o/8FqA/DLa4PVGNllCqINbZCcyomIUFV2CXL6dGbG91H1Gj6v31SOQm9QVXpHnPFgUiZyCc4FDh2UuhhAtK4EOM3S5JfqukB+Da9CovBdj4l65CB/rtzAjvnBMLYm1t30zC5Kz9SJgqb23fQ7TS07MLizInbQt+H2oEWZglxPDLA07vqD4UoFhXQx5cAUDF6+Y4bPVdKbOuEgO8+u0G+fME6VH2nokjXwhymi7pEKlNaC069PpPKfwk='}}]


## Key Observations

**Two LLM Calls:**
1. First call: Decide to use tool → Returns AIMessage with tool_calls
2. Second call: Generate final response → Returns AIMessage with content

**State Grows (Never Shrinks):**
- Each node appends messages
- Full history preserved
- Final state has 4 messages

**Router Controls Flow:**
- After Agent #1: tool_calls present → Route to tools
- After Agent #2: no tool_calls → Route to END

**LLM's Autonomy:**
- ✅ Identified correct tool from docstring
- ✅ Extracted parameters from natural language
- ✅ Decided when to stop (no more tools needed)
- ✅ Generated natural language response

**No Hardcoded Logic:**
- You defined: tools + graph structure
- LLM orchestrated: tool selection + execution flow

---

## ✅ Single Tool Execution Complete!

**Learned:**
- ✅ Complete execution flow: START → Agent → Tools → Agent → END
- ✅ Two LLM calls: (1) Decide to use tool, (2) Generate final response
- ✅ Message types: HumanMessage → AIMessage(tool_calls) → ToolMessage → AIMessage(content)
- ✅ State evolution: 1 message → 2 → 3 → 4 (grows with each node)
- ✅ Router logic: Checks tool_calls to determine next step
- ✅ LLM autonomy: Selects tool, extracts parameters, decides when done

**Critical Insights:**
- `llm_with_tools.invoke()` sends messages + tool schemas to Gemini
- LLM returns AIMessage with tool_calls (not the actual result)
- ToolNode executes the actual Python function
- Second LLM call sees the tool result and generates response
- No hardcoded logic - LLM orchestrates everything

**Next:** Notebook 5 - Sequential & Parallel Execution (multi-tool orchestration)